In [ ]:
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
from anndata import AnnData
import anndata as ad
import skimage
import matplotlib.pyplot as plt
from pathlib import Path


# import tangram for spatial deconvolution
import tangram as tg

sc.logging.print_header()
print(f"squidpy=={sq.__version__}")
print(f"tangram=={tg.__version__}")

# Including PKD samples (Muto24)

In [ ]:
atlas = sc.read_h5ad("/exports/archive/hg-groep-peters/snRNAseq_Cesare_Claudio_Early_Mod_Severe/snRNAseq/MKA_Extended/mka_extended_pkd_innerjoin.h5ad")

In [ ]:
library_id = 'pkd_3'

In [ ]:
output_dir = f'/exports/humgen/cnovellarausell/SevtapSpatial/outputs/tangram_outputs_with_PKD/{library_id.lower()}_v1/'

In [ ]:
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
adata_sc = sc.read_h5ad("/exports/archive/hg-groep-peters/snRNAseq_Cesare_Claudio_Early_Mod_Severe/snRNAseq/MKA_Extended/mka_extended_pkd_innerjoin_normalised_expression.h5ad")
adata_st = sc.read_h5ad("/exports/humgen/cnovellarausell/SevtapSpatial/h5ad/spatial_processed.h5ad")
adata_st = adata_st[adata_st.obs.library_id == library_id].copy()
img = sq.im.ImageContainer.load('/exports/humgen/cnovellarausell/SevtapSpatial/zarr/full{}_segmented_Cellpose_0.8_1'.format(library_id))

In [ ]:
adata_sc = adata_sc[~adata_sc.obs.Predicted_Celltype.isin(['Unknown'])].copy()

In [ ]:
adata_st.X = adata_st.layers['LogNorm'].copy()

In [ ]:
sq.im.calculate_image_features(
    adata_st,
    img,
    layer =  library_id,
    library_id = library_id,
    features="segmentation",
    key_added="segmentation_features",
    features_kwargs={
        "segmentation": {
            "label_layer": "segmented_custom",
            "props": ["label", "centroid"]
        }
    },
    mask_circle=True,
    n_jobs=3,
    show_progress_bar = True    
    )

In [ ]:
adata_st.obs["cell_count"] = adata_st.obsm["segmentation_features"]["segmentation_label"] # number of nuclei per voxel

In [ ]:
adata_st.obs["cell_count"].sum()

In [ ]:
adata_sc

In [ ]:
adata_sc.obs.groupby(['Predicted_Celltype']).Origin.value_counts().to_csv('celltype_origin.csv')

In [ ]:
def ct_adata(adata, obs_column):
    adata_dict = {}
    for i in set(adata.obs[obs_column]):
         adata_dict[i] = adata[adata.obs[obs_column] == i].copy()
    return adata_dict

def subsample_atlas(adata_dict, proportions, nr_cells):
    proportions.sort_values('Fraction', axis=0, ascending=False, inplace=True)
    
    # Filter out cell types with only one sample in the original adata_dict
    adata_dict = {ct: adata for ct, adata in adata_dict.items() if len(adata) > 1}
    
    for cell_type in proportions.index.to_list():
        if cell_type not in adata_dict:
            print(f"Skipping {cell_type} because it contains only one sample in the original data.")
            continue
        
        obs_to_sample = round(proportions.loc[cell_type]['Fraction'] * nr_cells)
        
        # Skip sampling if the number to sample is 1 or less
        if obs_to_sample <= 1:
            print(f"Skipping {cell_type} because the calculated sample size is {obs_to_sample}.")
            continue

        available_cells = len(adata_dict[cell_type])
        
        if available_cells >= obs_to_sample:
            sc.pp.subsample(adata_dict[cell_type], n_obs=obs_to_sample, random_state=0, copy=False)
        else:
            print(f"{cell_type}: Only {available_cells} available, but {obs_to_sample} requested.")
    
    return ad.concat(adata_dict.values())


In [ ]:
def subsample_atlas_excluding_origins(adata_dict, proportions, nr_cells, exclude_origins=("Muto24", "Humphreys23")):
    """
    Subsample cells from each cell type dictionary entry (adata_dict),
    but exclude cells whose .obs['Origin'] is in exclude_origins.
    
    Parameters
    ----------
    adata_dict : dict[str, anndata.AnnData]
        A dictionary keyed by cell-type, each value is an AnnData with cells of that type.
    proportions : pd.DataFrame
        DataFrame with index as cell types and a 'Fraction' column indicating the 
        fraction of total cells we want for each cell type.
    nr_cells : int
        Total number of cells to sample across all cell types.
    exclude_origins : tuple[str], optional
        Origins to exclude (default: ("Muto24", "Humphreys23")).

    Returns
    -------
    anndata.AnnData
        An AnnData object concatenated from the newly subsampled cell types.
    """

    # Sort proportions by highest Fraction first (so bigger cell types are processed first)
    proportions.sort_values('Fraction', axis=0, ascending=False, inplace=True)

    # 1) Exclude Muto24 / Humphreys23 from each AnnData
    #    and build a new dictionary with those filtered data
    filtered_adata_dict = {}
    for cell_type, adata in adata_dict.items():
        # Exclude rows with .obs['Origin'] in the specified tuple
        keep_mask = ~adata.obs['Origin'].isin(exclude_origins)
        filtered_adata = adata[keep_mask].copy()
        
        # Keep only if at least 1 cells remain
        if len(filtered_adata) < 1:
            print(f"Skipping {cell_type} because after excluding {exclude_origins}, fewer than 2 cells remain.")
        else:
            filtered_adata_dict[cell_type] = filtered_adata

    # 2) Filter out cell types that have only 1 sample in the original filtered dictionary
    filtered_adata_dict = {ct: adata for ct, adata in filtered_adata_dict.items()
                           if len(adata) > 1}

    # 3) Subsampling loop
    for cell_type in proportions.index.to_list():
        if cell_type not in filtered_adata_dict:
            print(f"Skipping {cell_type} because it has <2 cells after excluding {exclude_origins} or was not found.")
            continue
        
        obs_to_sample = round(proportions.loc[cell_type]['Fraction'] * nr_cells)
        
        # Skip sampling if the number to sample is 1 or less
        if obs_to_sample <= 1:
            print(f"Skipping {cell_type} because the calculated sample size is {obs_to_sample}.")
            continue

        available_cells = len(filtered_adata_dict[cell_type])

        if available_cells >= obs_to_sample:
            sc.pp.subsample(filtered_adata_dict[cell_type],
                            n_obs=obs_to_sample,
                            random_state=0,
                            copy=False)
        else:
            print(f"{cell_type}: Only {available_cells} available, but {obs_to_sample} requested.")

    # 4) Concatenate all subsampled AnnData objects
    if filtered_adata_dict:
        return ad.concat(filtered_adata_dict.values(), merge='same')
    else:
        print("No cell types remain after filtering and subsampling.")
        return None

In [ ]:
adata_dict = ct_adata(adata_sc, 'Predicted_Celltype')

In [ ]:
proportions = pd.read_csv(f'/exports/humgen/cnovellarausell/SevtapSpatial/cytospace/{library_id.lower()}_withatlas_cell_fractions_ZimmermanHumphreysMuto_highres_remapped_noint.tsv', sep='\t', index_col=0)

In [ ]:
proportions = proportions.T

In [ ]:
mapping = {
    'DTL ATL': 'DTL-ATL',
    'DCT CNT': 'DCT-CNT',
    'Glom Endo': 'Glom-Endo',
    'CD Trans': 'CD-Trans',
    'T3 Follicular B lymph': 'T3/Follicular B lymph',
    'CD4  T regs': 'CD4+ T regs',
    'CD4  T lymph': 'CD4+ T lymph',
    'CD4  Th17': 'CD4+ Th17',
    'Spp1  Resident Macrophages ':'Spp1+ Resident Macrophages ',
    'Gzma  CD8  T lymph': 'Gzma+ CD8+ T lymph',
    'Mrc1  Resident Macrophages':'Mrc1+ Resident Macrophages',
    'Gzma  NK':'Gzma+ NK',
    'PT AcInj': 'PT-AcInj',
    'PT R': 'PT-R',
    'PT FR': 'PT-FR',
    'PT Inj': 'PT-Inj',
    'Trans PTC': 'Trans-PTC'}

proportions.rename(index=mapping, inplace=True)

In [ ]:
adata_sc.obs.Origin.value_counts()

In [ ]:
adata_sc = subsample_atlas(adata_dict, proportions, adata_st.obs["cell_count"].sum())

In [ ]:
adata_sc

In [ ]:
adata_sc.X = adata_sc.X * 100000

In [ ]:
adata_sc

In [ ]:
sc.tl.rank_genes_groups(adata_sc, groupby="Predicted_Celltype", use_raw=False, method='wilcoxon')

In [ ]:
markers_df = pd.DataFrame(adata_sc.uns["rank_genes_groups"]["names"]).iloc[0:100, :]

In [ ]:
genes_sc = np.unique(markers_df.melt().value.values)
genes_st = adata_st.var_names.values
genes = list(set(genes_sc).intersection(set(genes_st)))

In [ ]:
tg.pp_adatas(adata_sc, adata_st, genes=genes)

In [ ]:
ad_map = tg.map_cells_to_space(
    adata_sc,
    adata_st,
    mode="constrained",
    target_count=adata_st.obs.cell_count.sum(),
    density_prior=np.array(adata_st.obs.cell_count) / adata_st.obs.cell_count.sum(),
    num_epochs=1000,
    device="cpu"
)

In [ ]:
ad_map.write_h5ad(output_dir + f"probs_constrained_denoised_{library_id}_subsampled_zimmermanhumphreysmuto_highres.h5ad")

In [ ]:
ad_map = sc.read_h5ad(output_dir + f"probs_constrained_denoised_{library_id}_subsampled_zimmermanhumphreysmuto_highres.h5ad")

In [ ]:
adata_sc.write_h5ad(output_dir + f"adata_sc_constrained_denoised_{library_id}_subsampled_zimmermanhumphreysmuto_highres.h5ad")

In [ ]:
adata_sc = sc.read_h5ad(output_dir + f"adata_sc_constrained_denoised_{library_id}_subsampled_zimmermanhumphreysmuto_highres.h5ad")

In [ ]:
tg.project_cell_annotations(ad_map, adata_st, annotation="Predicted_Celltype")
adata_st.obs = pd.concat([adata_st.obs, adata_st.obsm["tangram_ct_pred"]], axis=1)
adata_st.obsm['image_features'] = adata_st.obsm['segmentation_features']

In [ ]:
ad_ge = tg.project_genes(adata_map=ad_map, adata_sc=adata_sc)
df_all_genes = tg.compare_spatial_geneexp(ad_ge, adata_st, adata_sc)
tg.plot_auc(df_all_genes);
plt.savefig(output_dir + f"auc_constrained_denoised_{library_id}_subsampled_zimmermanhumphreysmuto_highres.svg")

In [ ]:
tg.create_segment_cell_df(adata_st)

In [ ]:
tg.count_cell_annotations(
    ad_map,
    adata_sc,
    adata_st,
    annotation="Predicted_Celltype",
)

In [ ]:
adata_segment = tg.deconvolve_cell_annotations(adata_st)

In [ ]:
adata_segment.obs.to_csv(f"/exports/archive/hg-groep-peters/Spatial_Transcriptomics_Snowball_Sevtap/{library_id}_tangram_segmented_nuclei_data_withPKD.csv")

In [ ]:
pd.DataFrame(adata_st.obsm['tangram_spot_centroids']).to_csv(output_dir + f"/{library_id}_tangram_spot_centroids_denoised_ZimmermanHumphreysMuto.csv")
adata_st.obsm['tangram_ct_pred'].to_csv(output_dir +  f"/{library_id}_tangram_ct_pred_denoised_ZimmermanHumphreysMuto.csv")
adata_st.obsm['tangram_ct_count'].to_csv(output_dir +  f"/{library_id}_tangram_ct_count_denoised_ZimmermanHumphreysMuto.csv")
adata_st.obsm['segmentation_features'].to_csv(output_dir +  f"/{library_id}_segmentation_features_denoised_ZimmermanHumphreysMuto.csv")
adata_st.uns['tangram_cell_segmentation'].to_csv(output_dir +  f"/{library_id}_tangram_cell_segmentation_denoised_ZimmermanHumphreysMuto.csv")
df = adata_segment.obs.cluster.value_counts().rename_axis('cell_type').to_frame('counts')

In [ ]:
df.to_csv(output_dir + '{}_celltype_counts_denoised_zimmermanhumphreysmuto_highres.csv'.format(library_id))

In [ ]:
import pickle as pkl
palette = pkl.load( open( "/exports/archive/hg-groep-peters/Healthy_Mouse_Atlas_Claudio/KidneyAtlas/predicted_celltype_palette_highres_2.pkl", "rb" ) )

In [ ]:
palette['Myofib']  = '#963b00'
palette['CD4+ T lymph'] = '#c4bfeb'
palette['Gzma+ CD8+ T lymph'] = '#88bf35'
palette['Memory B lymph'] = '#a691b4'
palette['NKT1'] = '#01a7f4'
palette['Trans-PTC'] = '#321fa0'
palette['PT-FR'] = '#03655c'
palette['PT-R'] = '#059e3f'
palette['PT-AcInj'] = '#615318'
palette['PT-Inj'] = '#fc362f'

In [ ]:
import re
import os

def clean_filename(filename):
    """Clean filename to remove problematic characters"""
    filename = str(filename)
    filename = filename.replace(' ', '_')   # Replace spaces
    filename = filename.replace('/', '_')   # Replace forward slashes  
    filename = filename.replace('\\', '_')  # Replace backslashes
    filename = re.sub(r'[^\w\-_.]', '_', filename)  # Replace other problematic chars
    filename = re.sub(r'_+', '_', filename)         # Replace multiple underscores
    filename = filename.strip('_')                  # Remove leading/trailing underscores
    return filename


# Create a reference plot with ALL clusters to get the axis limits
print("Creating reference plot with all clusters...")
fig_ref, ax_ref = plt.subplots(1, 1, figsize=(20, 20))
sc.pl.spatial(
    adata_segment,
    library_id=library_id,
    color="cluster",
    size=0.5,
    show=False,
    palette=palette,
    frameon=True,
    alpha_img=0.3,
    ax=ax_ref,
)

# Get the axis limits from the full plot
xlim = ax_ref.get_xlim()
ylim = ax_ref.get_ylim()
print(f"Reference plot limits - X: {xlim}, Y: {ylim}")
plt.close(fig_ref)

# Plot the combined Trans-PTC and FR-PT clusters
print("Creating combined Trans-PTC and FR-PT plot...")
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
sc.pl.spatial(
    adata_segment[adata_segment.obs.cluster.isin(['Trans-PTC', 'PT-FR'])],
    library_id=library_id,
    color="cluster",
    size=0.5,
    show=False,
    palette=palette,
    frameon=True,
    alpha_img=0.3,
    legend_fontsize=20,
    ax=ax,
)


# Apply the fixed limits from the reference plot
ax.set_xlim(xlim)
ax.set_ylim(ylim)

output_path = os.path.join(
    output_dir,
    f"{library_id}_Trans-PTC_FR-PT_combined_denoised_zimmermanhumphreysmuto_highres_noint_original.pdf"
)
plt.savefig(output_path)
plt.close(fig)
print(f"Saved combined plot: {output_path}")

# Now loop over individual clusters with fixed limits
print("Creating individual cluster plots...")
for cluster in adata_segment.obs['cluster'].unique():
    cluster_clean = clean_filename(cluster)
    #print(f"Processing cluster: {cluster} -> {cluster_clean}")
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    sc.pl.spatial(
        adata_segment[adata_segment.obs.cluster == cluster],
        library_id=library_id,
        color="cluster",
        size=0.5,
        show=False,
        palette=palette,
        frameon=True,
        alpha_img=0.3,
        legend_fontsize=20,
        ax=ax,
    )
    
    # Apply the fixed limits from the reference plot
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    output_path = os.path.join(
        output_dir,
        f"{library_id}_{cluster_clean}_denoised_zimmermanhumphreysmuto_highres_noint_original.pdf"
    )
    plt.savefig(output_path)
    plt.close(fig)
    
    print(f"Saved: {output_path}")

print("All plots completed!")